<a href="https://colab.research.google.com/github/saishshinde15/CrewAI/blob/main/Chatting(agent)_with_multiple_websites.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install crewai langchain-community

In [32]:


import warnings
warnings.filterwarnings('ignore')



In [31]:
%pip install -qU langchain_community pypdf

In [3]:
%pip install -qU langchain-text-splitters

In [4]:
%pip install -qU langchain-huggingface

In [5]:
#%pip install -qU langchain-docling

In [6]:
%pip install -qU langchain_community beautifulsoup4

In [7]:

import os
from huggingface_hub import notebook_login

notebook_login()


In [8]:
from crewai import Agent, Crew, Task,LLM, Process

## Retrival 1

In [9]:
%pip install -qU  nest_asyncio

# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()

In [10]:
from langchain_community.document_loaders import WebBaseLoader

In [11]:
loader = WebBaseLoader(["https://python.langchain.com/docs/concepts/", "https://langchain-ai.github.io/langgraph/concepts/#langgraph"])
loader.requests_per_second = 1
docs = loader.aload()
docs

<ipython-input-11-407e76b9751f>:3: LangChainDeprecationWarning: See API reference for updated usage: https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html
  docs = loader.aload()
Fetching pages: 100%|##########| 2/2 [00:00<00:00, 18.93it/s]


[Document(metadata={'source': 'https://python.langchain.com/docs/concepts/', 'title': 'Conceptual guide | 🦜️🔗 LangChain', 'description': 'This guide provides explanations of the key concepts behind the LangChain framework and AI applications more broadly.', 'language': 'en'}, page_content='\n\n\n\n\nConceptual guide | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as

## should clean the above data for better results

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [13]:
textsplitter1=RecursiveCharacterTextSplitter(chunk_size=1500,chunk_overlap=150)
docs_main_1=textsplitter1.split_documents(docs)


In [14]:
print(docs_main_1[7])

page_content='This guide provides explanations of the key concepts behind the LangChain framework and AI applications more broadly.
We recommend that you go through at least one of the Tutorials before diving into the conceptual guide. This will provide practical context that will make it easier to understand the concepts discussed here.
The conceptual guide does not cover step-by-step instructions or specific implementation examples — those are found in the How-to guides and Tutorials. For detailed reference material, please see the API reference.
High level​' metadata={'source': 'https://python.langchain.com/docs/concepts/', 'title': 'Conceptual guide | 🦜️🔗 LangChain', 'description': 'This guide provides explanations of the key concepts behind the LangChain framework and AI applications more broadly.', 'language': 'en'}


In [15]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [16]:
pip install -qU langchain-community faiss-cpu

In [17]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [18]:
vectorstore=FAISS.from_documents(docs_main_1,embeddings)

In [19]:
%pip install -qU langchain-google-genai

In [42]:
from google.colab import userdata
gemini_key=userdata.get('gemini')

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    api_key=gemini_key
)

In [21]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())
query = "What is LangChain?"
qa.invoke(query)

{'query': 'What is LangChain?',
 'result': 'LangChain is a framework for developing applications powered by large language models (LLMs).  It provides tools and interfaces to help organize and manage the different components needed to build these applications.'}

# Creating custom tool

In [22]:
from crewai.tools import tool

@tool("LangChain QA Tool")
def langchain_qa(question: str) -> str:
    """Answers questions about LangChain using a pre-built QA chain."""
    try:
        answer = qa.invoke(question)
        return answer
    except Exception as e:
        return f"An error occurred: {e}"

In [23]:
llm = LLM(
    model="gemini/gemini-1.5-flash",
    temperature=0.7,
    api_key=gemini_key
)


In [24]:
chat_agent = Agent(
    role="Senior Chat Support Representative",
    goal="Be the most friendly and helpful and answer user questions properly and meaningfully.",
    backstory=(
        "You are a highly skilled and professional support agent equipped to interact with high-level representatives. "
        "Your role is to provide meaningful, well-informed, and articulate responses to complex queries. "
        "As a vital part of the support team, you are expected to maintain professionalism, clarity, and precision in all interactions. "
        "Your goal is to ensure every conversation reflects a high standard of service and leaves a positive impression."
    ),
    tools=[langchain_qa],
    llm=llm,
    verbose=True,
    allow_delegation=False
)



In [25]:
inquiry_resolution = Task(
    description=(
        "A high-level representative just reached out with the following query:\n"
        "{input}\n\n"
        "This is a critical inquiry, and it is your responsibility to utilize your knowledge and resources "
        "to provide the most accurate and comprehensive response possible. "
        "Your goal is to maintain professionalism, clarity, and precision in addressing their question, "
        "ensuring they receive a detailed and meaningful solution."
    ),
    expected_output=(
        "A thorough and well-structured response that directly addresses the query, "
        "provides actionable insights, and demonstrates a deep understanding of the topic. "
        "The response should include references to relevant information, tools, or external resources used, "
        "and maintain a professional and respectful tone. "
        "Ensure the answer is complete, leaving no questions unanswered."
    ),
    agent=chat_agent,
    input_variables=["input"],
    verbose=True,

)


In [36]:
crew=Crew(
    agents=[chat_agent],
    tasks=[inquiry_resolution],
    process=Process.sequential,
    memory=True,
    #verbose=True,
    embedder={
        "provider": "google",
        "config": {
            "api_key": gemini_key,
            # Replacing model_name with model
            "model": "models/embedding-001"
        }
    }
)

In [37]:
user_input = {"input":"what is Integration tests in langchain?"}

In [38]:
result = crew.kickoff(inputs=user_input)

# Agent: Senior Chat Support Representative
## Task: A high-level representative just reached out with the following query:
what is Integration tests in langchain?

This is a critical inquiry, and it is your responsibility to utilize your knowledge and resources to provide the most accurate and comprehensive response possible. Your goal is to maintain professionalism, clarity, and precision in addressing their question, ensuring they receive a detailed and meaningful solution.


# Agent: Senior Chat Support Representative
## Thought: Thought: I need to use the LangChain QA Tool to get information about integration tests in LangChain.
## Using tool: LangChain QA Tool
## Tool Input: 
"{\"question\": \"What are integration tests in LangChain?\"}"
## Tool Output: 
{'query': 'What are integration tests in LangChain?', 'result': 'Integration tests in LangChain verify the correctness of the interaction between components.  They are usually run with access to the underlying API that powers an 

##### Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

###### Ignore the above message as this is due to rate limits of gemini

In [39]:
print(result)

In LangChain, integration tests are crucial for ensuring that different components of your application work seamlessly together. Unlike unit tests, which focus on individual components, integration tests examine the interactions between various parts, such as chains, LLMs, prompts, and memory components.  They verify that data flows correctly between these modules and that the overall system behaves as expected.  These tests are typically run with access to the APIs of the integrated components (e.g., an LLM API), simulating real-world usage scenarios.  This helps catch integration issues early in the development process, preventing unexpected behavior in production.  For example, an integration test might involve chaining an LLM with a vector database and verifying that the LLM correctly uses information retrieved from the database to generate a response.  Or, it could test the interaction between a chain and a memory component, ensuring that the memory component stores and retrieves 

## Follow up question(testing memory)

In [40]:
user_input = {"input":"can you be more clear?"}
result = crew.kickoff(inputs=user_input)

# Agent: Senior Chat Support Representative
## Task: A high-level representative just reached out with the following query:
can you be more clear?

This is a critical inquiry, and it is your responsibility to utilize your knowledge and resources to provide the most accurate and comprehensive response possible. Your goal is to maintain professionalism, clarity, and precision in addressing their question, ensuring they receive a detailed and meaningful solution.


# Agent: Senior Chat Support Representative
## Thought: Thought:The user's request "can you be more clear?" is vague. To provide a helpful response, I need to understand what aspect needs clarification.  I will assume the user is asking for clarification on the nature and importance of integration testing within the LangChain framework. I will use the LangChain QA Tool to get a foundational answer and then expand upon it to create a comprehensive response suitable for a high-level representative.
## Using tool: LangChain QA Too

In [41]:
print(result)

Integration testing in LangChain is crucial for ensuring the seamless interaction and correct functionality of various components within your application. Unlike unit tests, which focus on individual modules in isolation, integration tests examine how these modules interact and function together as a cohesive system.  In the context of LangChain, this involves verifying the interplay between various elements such as:

* **Chains:**  Testing the complete flow of data and logic across multiple chain steps, ensuring each step correctly processes the output of the previous step.
* **LLMs:** Validating that the chosen Large Language Model (LLM) integrates correctly and provides the expected responses within the chain. This includes verifying proper prompt formatting and handling of LLM-specific outputs.
* **Prompts:**  Ensuring prompts are correctly formatted and passed to the LLM, and that the LLM's responses are correctly interpreted and processed.
* **Memory:** Confirming that memory com

In [43]:
user_input = {"input":"now tell me wht is Time Travel in langgraph?"}
result = crew.kickoff(inputs=user_input)

# Agent: Senior Chat Support Representative
## Task: A high-level representative just reached out with the following query:
now tell me wht is Time Travel in langgraph?

This is a critical inquiry, and it is your responsibility to utilize your knowledge and resources to provide the most accurate and comprehensive response possible. Your goal is to maintain professionalism, clarity, and precision in addressing their question, ensuring they receive a detailed and meaningful solution.


# Agent: Senior Chat Support Representative
## Thought: Thought:I need to use the LangChain QA Tool to get information about Time Travel in LangGraph.  Since "LangGraph" isn't a standard LangChain term, I'll need to carefully phrase my question to get relevant information.  I suspect the user is referring to some form of state management or maintaining context across multiple interactions within a LangChain application.
## Using tool: LangChain QA Tool
## Tool Input: 
"{\"question\": \"How does LangChain h

In [44]:
print(result)

The term "Time Travel" in the context of LangChain likely refers to the ability to access and utilize previous interactions or turns within a conversation or chain. LangChain achieves this primarily through its various memory components. These components store and retrieve information from past interactions, enabling the chain or agent to maintain context and build upon previous exchanges.  There isn't a specific feature called "Time Travel," but the functionality is implemented through the following mechanisms:

* **ConversationBufferMemory:** This type of memory stores a buffer of recent conversation turns, allowing the model to access and refer to a limited history of the conversation.  The buffer size determines how much of the past conversation is accessible.

* **ConversationSummaryMemory:** This memory type summarizes the conversation history, providing a condensed overview of past interactions. This is useful for longer conversations where accessing the full history might be co